# [Lv4-Day4-Lab1] The Central Hub: Building a Production-Ready MCP Server

### 실습 목표
4일차의 첫 실습에서는, 3일차까지 만들었던 모든 Agent들이 공유하고 협업할 수 있는 **'중앙 자원 허브'**를 구축합니다. 우리는 더 이상 각 Agent가 개별적으로 Tool을 소유하는 방식이 아닌, 모든 Tool과 리소스를 **'서비스(Service)'** 형태로 제공하는 **MCP(Model Context Protocol) 서버**를 구현합니다. 이는 Agent 시스템을 개인 프로젝트 수준에서, 여러 팀과 서비스가 협력하는 **실무적인 분산 시스템 아키텍처**로 격상시키는 핵심적인 단계입니다.

이번 **Part 1**에서는 이 서버의 가장 기본적인 뼈대를 세우는 작업에 집중합니다.

1.  **FastAPI 프레임워크 이해:** 현대적인 고성능 웹 프레임워크인 `FastAPI`의 기본 구조와 장점(비동기, 자동 문서화, Pydantic 기반)을 이해합니다.
2.  **기본 서버 뼈대 구축:** `FastAPI`를 사용하여 "Hello World" 수준의 간단한 API 서버를 만들고, 서버의 상태를 확인할 수 있는 기본 엔드포인트(`/`)를 구현합니다.
3.  **서버 실행 및 노출:** `uvicorn`으로 FastAPI 서버를 실행하고, 접근 가능한 URL을 안전하게 확보하는 방법을 마스터합니다.

### 0. 사전 준비: 라이브러리 설치 및 신규 API 키 설정
이번 실습에서는 웹 서버 구축을 위한 `fastapi`, `uvicorn`, 그리고 새로운 뉴스 검색 Tool을 위한 `newsapi-python` 라이브러리를 새롭게 설치합니다. 또한, **News API**를 위한 API 키를 추가로 발급받습니다

In [ ]:
# !pip install fastapi uvicorn python-dotenv nest-asyncio aiohttp httpx newsapi-python arxiv tavily-python slowapi -q

In [ ]:
# !pip install langgraph crewai crewai_tools google-adk langchain-google-genai tavily-python wolframalpha langfuse pydantic instructor requests -q

In [ ]:
import os
from getpass import getpass

# os.environ.get()을 사용하여 이미 키가 설정되었는지 확인하고,
# 설정되지 않은 경우에만 getpass를 통해 입력을 요청합니다.

# GOOGLE_API_KEY 설정
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Google API 키를 입력하세요: ")
    # crewAI의 LiteLLM 호환성을 위한 설정
    os.environ["GEMINI_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# TAVILY_API_KEY 설정
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass("Tavily API 키를 입력하세요: ")

# NEWS_API_KEY 설정
if "NEWS_API_KEY" not in os.environ:
    os.environ["NEWS_API_KEY"] = getpass("News API 키를 입력하세요: ")

# ⚠️ [보안] 서버 인증을 위한 마스터 API 키를 직접 정의합니다.
# 실무에서는 더 복잡하고 안전한 방식으로 생성 및 관리해야 합니다.
os.environ["MASTER_API_KEY"] = "samsung-llm-agent-lv4-master-key"


print("✅ 모든 API 키가 성공적으로 설정되었습니다.")

### 1. 서버의 뼈대: FastAPI 기본 구조 구현

**Why FastAPI?** FastAPI는 Python 3.7+의 표준 타입 힌트를 기반으로 작동하는 매우 현대적인 고성능 웹 프레임워크입니다. 우리가 이 프레임워크를 선택한 이유는 다음과 같습니다:
- **고성능:** 비동기(Async)를 완벽하게 지원하여, 여러 Agent의 동시 요청을 효율적으로 처리할 수 있습니다.
- **개발 생산성:** Pydantic을 내장하여 데이터 유효성 검사를 자동으로 처리하며, 코드량이 매우 간결합니다.
- **자동 문서화:** 코드만 작성하면 **Swagger UI**와 **ReDoc**이라는 두 종류의 대화형 API 문서를 자동으로 생성해줍니다. 이는 다른 개발팀이나 Agent가 우리 서버의 기능을 쉽게 파악하고 연동할 수 있도록 돕는 매우 강력한 기능입니다.

먼저, `%%writefile`을 사용하여 가장 기본적인 서버 뼈대 코드를 `mcp_server.py` 파일로 저장합니다.

In [ ]:
%%writefile mcp_server.py

from fastapi import FastAPI, HTTPException, Depends
from pydantic import BaseModel, Field
import os

# --- FastAPI 앱 초기화 ---
app = FastAPI(
    title="LLM Agent Resource Hub (MCP Server)",
    description="다양한 Agent들이 공유할 수 있는 Tool과 리소스를 제공하는 중앙 MCP 서버입니다.",
    version="1.0.0"
)

# --- 기본 엔드포인트: 서버 상태 확인 ---
@app.get("/", summary="서버 상태 확인", tags=["Status"])
async def read_root():
    """서버가 정상적으로 실행 중인지 확인하는 기본 엔드포인트입니다."""
    return {"status": "ok", "message": "MCP Server is running successfully."}

print("✅ mcp_server.py 파일의 기본 뼈대가 생성되었습니다.")

### 2. 서버 실행 및 외부 노출
이제 방금 만든 `mcp_server.py` 파일을 웹 서버(`uvicorn`)를 통해 실행합니다

In [ ]:
import sys
import subprocess
import time
from IPython.display import display, HTML


def launch_fastapi_local(app_filename="mcp_server.py", port=8501):
    """
    FastAPI 앱(Uvicorn)을 백그라운드에서 실행하고 로컬 접속 URL을 생성합니다.
    """
    # 기존 Uvicorn 프로세스 정리
    try:
        subprocess.run(["pkill", "-f", "uvicorn"], capture_output=True)
        print("기존 FastAPI(Uvicorn) 프로세스를 정리했습니다.")
        time.sleep(2)
    except Exception:
        pass

    # FastAPI 앱(Uvicorn) 백그라운드 실행
    try:
        app_name = app_filename.replace(".py", "")
        command = [sys.executable, "-m", "uvicorn", f"{app_name}:app", "--host", "0.0.0.0", "--port", str(port)]
        server_process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"FastAPI 서버를 포트 {port}에서 시작하는 중...")
        time.sleep(5)  # 서버가 시작될 시간을 줍니다.
    except Exception as e:
        print(f"FastAPI 서버 시작에 실패했습니다: {e}")
        return None, None

    # 결과 출력
    local_url = f"http://localhost:{port}"
    print("\n" + "=" * 60)
    print("FastAPI 서버가 성공적으로 시작되었습니다!")
    print("=" * 60)
    print(f"다음 URL을 클릭하여 API 서버에 접속하세요:")
    print(f"🔗 {local_url}")

    try:
        display(HTML(f'<b><a href="{local_url}" target="_blank">👉 여기를 클릭하여 서버에 접속하세요</a></b>'))
    except NameError:  # IPython.display가 없는 환경
        pass

    return local_url, server_process


# --- 서버 실행 ---
server_url, server_process = launch_fastapi_local("mcp_server.py", 8501)

if server_url:
    print(f"\n✅ 서버가 성공적으로 시작되었습니다. Local URL: {server_url}")
    print(f"   - 서버 프로세스 ID: {server_process.pid}")
else:
    print("\n❌ 설정 중 문제가 발생했습니다. 에러 메시지를 확인하고 셀을 다시 실행해보세요.")

이제 FastAPI의 자동 생성 문서도 확인해보세요:

Swagger UI: {your_url}/docs
ReDoc: {your_url}/redoc

이 문서들을 통해 API 구조를 시각적으로 확인하고, 나중에 다른 Agent들이 이 서버와 통신할 때 참조할 수 있습니다.

### 3. 서버 상태 확인 및 자동 문서 테스트
서버가 성공적으로 실행되었다면, `requests` 라이브러리를 사용하여 우리가 만든 기본 엔드포인트(`/`)에 프로그래밍적으로 요청을 보내보고, 정상적인 응답이 오는지 확인합니다. 또한, FastAPI의 강력한 기능인 자동 생성 문서를 확인하는 방법을 알아봅니다.

In [ ]:
import requests
import time
from IPython.display import display, HTML

if "server_url" in locals() and server_url:
    # 서버가 완전히 시작될 때까지 잠시 대기
    time.sleep(3)

    try:
        response = requests.get(server_url)
        response.raise_for_status()  # 200번대 상태 코드가 아니면 에러 발생

        print("--- API 응답 확인 ---")
        print(f"Status Code: {response.status_code}")
        print(f"Response JSON: {response.json()}")

        # 자동 생성된 API 문서 링크 제공
        swagger_url = f"{server_url}/docs"
        redoc_url = f"{server_url}/redoc"

        print("\n--- 자동 생성된 API 문서 ---")
        display(HTML(f'<li><b>Swagger UI (대화형):</b> <a href="{swagger_url}" target="_blank">{swagger_url}</a></li>'))
        display(HTML(f'<li><b>ReDoc (문서형):</b> <a href="{redoc_url}" target="_blank">{redoc_url}</a></li>'))

    except requests.exceptions.RequestException as e:
        print(f"❌ 서버에 연결할 수 없습니다: {e}")
        print("   이전 셀의 서버 실행 로그에 오류가 없는지 확인해주세요.")
else:
    print("❌ 서버 URL이 생성되지 않았습니다. 이전 셀을 다시 실행해주세요.")

### 4. 서버의 능력 (Part 1): Tool 엔드포인트 구현

이제 우리 MCP 서버의 핵심 기능인 **Tool 서비스**를 구현합니다. 3일차에 우리 Agent의 '개인 Toolbox'에 있던 Tool들을 이제 모든 Agent가 네트워크를 통해 접근할 수 있는 **'공용 장비실'**로 옮기는 과정입니다. 이는 코드의 재사용성을 극대화하고, 각 Agent의 책임을 명확히 분리하는 서비스 지향 아키텍처(Service-Oriented Architecture)의 핵심입니다.

**구현 내용:**
1.  **Tool 로직 모듈화:** `%%writefile`을 사용하여 Tool들의 실제 실행 로직을 `server_tools.py`라는 별도의 파일로 분리하여 서버 코드의 가독성과 유지보수성을 높입니다.
2.  **신규 Tool 추가:** 커리큘럼에 따라, 학술 논문 검색을 위한 `arxiv_search`와 최신 뉴스 기사 검색을 위한 `news_api` Tool을 새롭게 구현합니다.
3.  **FastAPI 엔드포인트 생성:** `mcp_server.py`에 `/tools/{tool_name}` 형태의 엔드포인트를 추가하고, Pydantic 모델을 사용하여 각 Tool의 입력값을 검증합니다.

In [ ]:
%%writefile server_tools.py

import os
from tavily import TavilyClient
from newsapi import NewsApiClient
import arxiv

# --- 1. 웹 검색 (Tavily) ---
async def web_search(query: str) -> str:
    try:
        client = TavilyClient(api_key=os.environ['TAVILY_API_KEY'])
        response = client.search(query=query, max_results=3, search_depth="advanced")
        return str([{"title": obj['title'], "url": obj['url'], "content": obj['content']} for obj in response['results']])
    except Exception as e:
        return f"Tavily API 오류: {e}"

# --- 2. 뉴스 기사 검색 (NewsAPI) ---
async def news_api_search(query: str) -> str:
    try:
        client = NewsApiClient(api_key=os.environ['NEWS_API_KEY'])
        response = client.get_everything(q=query, language='ko', sort_by='relevancy', page_size=3)
        if response['status'] == 'ok':
            return str([{"title": article['title'], "url": article['url'], "description": article['description']} for article in response['articles']])
        else:
            return f"News API 오류: {response.get('message', 'Unknown error')}"
    except Exception as e:
        return f"News API 클라이언트 오류: {e}"

# --- 3. 학술 논문 검색 (Arxiv) ---
async def arxiv_search(query: str) -> str:
    try:
        client = arxiv.Client()
        search = arxiv.Search(query=query, max_results=2, sort_by=arxiv.SortCriterion.Relevance)
        results = list(client.results(search))
        return str([{"title": result.title, "authors": [str(a) for a in result.authors], "summary": result.summary, "pdf_url": result.pdf_url} for result in results])
    except Exception as e:
        return f"Arxiv 검색 오류: {e}"

In [ ]:
%%writefile -a mcp_server.py

from fastapi import Body
from typing import Dict, Any
# 방금 만든 Tool 함수들을 import
from server_tools import web_search, news_api_search, arxiv_search

# --- Pydantic 모델 정의: Tool 호출을 위한 입력 스키마 ---
class ToolCallRequest(BaseModel):
    query: str = Field(..., description="Tool에 전달할 검색어 또는 입력값")

# --- Tool 엔드포인트 구현 ---
@app.post("/tools/web_search", summary="일반 웹 검색 수행", tags=["Tools"])
async def run_web_search(request: ToolCallRequest):
    result = await web_search(request.query)
    return {"tool": "web_search", "query": request.query, "result": result}

@app.post("/tools/news_api", summary="최신 뉴스 기사 검색", tags=["Tools"])
async def run_news_search(request: ToolCallRequest):
    result = await news_api_search(request.query)
    return {"tool": "news_api", "query": request.query, "result": result}

@app.post("/tools/arxiv_search", summary="학술 논문 검색", tags=["Tools"])
async def run_arxiv_search(request: ToolCallRequest):
    result = await arxiv_search(request.query)
    return {"tool": "arxiv_search", "query": request.query, "result": result}

print("✅ 3개의 Tool 엔드포인트가 mcp_server.py에 추가되었습니다.")

### 5. 서버의 능력 (Part 2): 정적 리소스 엔드포인트 구현

Tool과 같이 동적인 기능 외에도, Agent들이 공통적으로 참고해야 할 정적인 데이터(블로그 템플릿, 스타일 가이드 등)를 제공하는 것도 MCP 서버의 중요한 역할입니다. 이러한 데이터를 제공하는 리소스 엔드포인트를 구현합니다.

In [ ]:
%%writefile server_resources.py

BLOG_TEMPLATES = {
    "tech_analysis": "## 제목\n\n### 1. 기술 개요\n\n### 2. 핵심 작동 원리...",
    "product_review": "## 제목\n\n### 1. 첫인상 및 디자인\n\n### 2. 주요 기능..."
}

STYLE_GUIDES = {
    "default": "문체: 전문적이면서도 명확하게.\n대상 독자: 기술에 관심 있는 일반인.\n어조: 객관적이고 사실 기반.",
    "samsung_newsroom": "문체: 삼성전자 뉴스룸 공식 톤앤매너.\n대상 독자: 언론인 및 IT 업계 종사자.\n어조: 신뢰감을 주는 공식적인 어조."
}

async def get_template(template_id: str):
    template = BLOG_TEMPLATES.get(template_id)
    if template:
        return template
    else:
        return {"error": "Template not found"}

async def get_style_guide(guide_id: str):
    guide = STYLE_GUIDES.get(guide_id)
    if guide:
        return guide
    else:
        return {"error": "Style guide not found"}

In [ ]:
%%writefile -a mcp_server.py

from server_resources import get_template, get_style_guide

# --- 리소스 엔드포인트 구현 ---
@app.get("/resources/templates/{template_id}", summary="블로그 템플릿 조회", tags=["Resources"])
async def read_template(template_id: str):
    content = await get_template(template_id)
    if "error" in content:
        raise HTTPException(status_code=404, detail=content["error"])
    return {"resource": "template", "id": template_id, "content": content}

@app.get("/resources/style_guides/{guide_id}", summary="스타일 가이드 조회", tags=["Resources"])
async def read_style_guide(guide_id: str):
    content = await get_style_guide(guide_id)
    if "error" in content:
        raise HTTPException(status_code=404, detail=content["error"])
    return {"resource": "style_guide", "id": guide_id, "content": content}

print("✅ 2개의 리소스 엔드포인트가 mcp_server.py에 추가되었습니다.")

### 6. 서버 재실행 및 기능 테스트

새로운 기능들이 추가된 서버를 다시 실행하고, `requests`를 사용하여 각 엔드포인트가 의도대로 잘 작동하는지 테스트합니다. 또한, FastAPI의 자동 문서(`/docs`)가 어떻게 업데이트되었는지 확인해보세요. 새로운 `Tools`와 `Resources` 섹션이 추가되고, 각 엔드포인트에 대한 상세한 명세와 테스트 기능을 제공하는 것을 볼 수 있습니다.

In [ ]:
# 이전 셀들에서 정의된 launch_fastapi_local 함수를 사용하여 서버를 재실행합니다.
server_url, server_process = launch_fastapi_local("mcp_server.py", 8501)

In [ ]:
if server_url:
    print(f"\n기능이 추가된 서버가 성공적으로 재시작되었습니다. Local URL: {server_url}")
    # 테스트를 위해 잠시 대기
    import time

    time.sleep(5)

    # --- Tool 엔드포인트 테스트 ---
    print("\n--- Tool 엔드포인트 테스트 중... ---")
    try:
        test_query = "NVIDIA Blackwell Architecture"
        response = requests.post(f"{server_url}/tools/arxiv_search", json={"query": test_query})
        response.raise_for_status()
        print(f"Arxiv Search Tool 테스트 성공! (결과 일부: {str(response.json())[:200]}...)")
    except Exception as e:
        print(f"Arxiv Search Tool 테스트 실패: {e}")

    # --- 리소스 엔드포인트 테스트 ---
    print("\n--- 리소스 엔드포인트 테스트 중... ---")
    try:
        response = requests.get(f"{server_url}/resources/templates/tech_analysis")
        response.raise_for_status()
        print(f"Template Resource 테스트 성공! (결과: {response.json()['content'][:30]}...)")
    except Exception as e:
        print(f"Template Resource 테스트 실패: {e}")
else:
    print("\n서버 재시작에 실패했습니다. 이전 셀의 로그를 확인해주세요.")

### 7. 서버의 갑옷 (Part 1): API 키 인증 구현

현재 우리 서버는 URL만 알면 누구나 접근하여 우리의 API 키(Tavily, NewsAPI 등)를 무제한으로 사용할 수 있는 매우 위험한 상태입니다. 실무 환경에서는 허가된 클라이언트(Agent)만이 서버를 사용할 수 있도록 **인증(Authentication)** 체계를 구축하는 것이 필수적입니다.

`FastAPI`의 강력한 **의존성 주입(Dependency Injection)** 시스템을 사용하여, 모든 요청이 `X-API-Key`라는 HTTP 헤더에 올바른 마스터 API 키를 포함하고 있는지 검증하는 보안 계층을 추가합니다. 올바른 키가 없는 요청은 `401 Unauthorized` 오류를 반환하며 즉시 차단됩니다.

In [ ]:
%%writefile -a mcp_server.py

from fastapi import Security, Request
from fastapi.security import APIKeyHeader

# --- 보안: API 키 인증 ---
API_KEY_NAME = "X-API-Key"
api_key_header = APIKeyHeader(name=API_KEY_NAME, auto_error=True)

async def get_api_key(api_key: str = Security(api_key_header)):
    """요청 헤더에서 API 키를 가져와 마스터 키와 비교하는 의존성 함수"""
    if api_key == os.environ["MASTER_API_KEY"]:
        return api_key
    else:
        raise HTTPException(
            status_code=401,
            detail="Invalid or missing API Key",
        )

# --- 기존 엔드포인트를 보안이 적용된 버전으로 교체 ---
# 새로운 경로로 보안 엔드포인트 생성

@app.post("/secure/tools/web_search", summary="일반 웹 검색 수행 (보안)", tags=["Protected Tools"])
async def secure_web_search(request: ToolCallRequest, api_key: str = Depends(get_api_key)):
    result = await web_search(request.query)
    return {"tool": "web_search", "query": request.query, "result": result}

@app.post("/secure/tools/news_api", summary="최신 뉴스 기사 검색 (보안)", tags=["Protected Tools"])
async def secure_news_search(request: ToolCallRequest, api_key: str = Depends(get_api_key)):
    result = await news_api_search(request.query)
    return {"tool": "news_api", "query": request.query, "result": result}

@app.post("/secure/tools/arxiv_search", summary="학술 논문 검색 (보안)", tags=["Protected Tools"])
async def secure_arxiv_search(request: ToolCallRequest, api_key: str = Depends(get_api_key)):
    result = await arxiv_search(request.query)
    return {"tool": "arxiv_search", "query": request.query, "result": result}

@app.get("/secure/resources/templates/{template_id}", summary="블로그 템플릿 조회 (보안)", tags=["Protected Resources"])
async def secure_read_template(template_id: str, api_key: str = Depends(get_api_key)):
    content = await get_template(template_id)
    if isinstance(content, dict) and "error" in content:
        raise HTTPException(status_code=404, detail=content["error"])
    return {"resource": "template", "id": template_id, "content": content}

@app.get("/secure/resources/style_guides/{guide_id}", summary="스타일 가이드 조회 (보안)", tags=["Protected Resources"])
async def secure_read_style_guide(guide_id: str, api_key: str = Depends(get_api_key)):
    content = await get_style_guide(guide_id)
    if isinstance(content, dict) and "error" in content:
        raise HTTPException(status_code=404, detail=content["error"])
    return {"resource": "style_guide", "id": guide_id, "content": content}

print("✅ 보안 API 키 인증이 적용된 새로운 엔드포인트들이 추가되었습니다.")

### 8. 서버의 갑옷 (Part 2): 요청 수 제한 (Rate Limiting) 구현

인증을 통과한 Agent라도, 버그로 인해 무한 루프에 빠져 1초에 수백 번씩 우리 서버에 요청을 보낼 수 있습니다. 이는 서버를 다운시키거나 외부 API(Tavily 등) 비용을 순식간에 소진시키는 심각한 문제를 야기할 수 있습니다.

**요청 수 제한(Rate Limiting)**은 이러한 위험을 방지하는 핵심적인 안정성 장치입니다. `slowapi` 라이브러리를 사용하여, 특정 클라이언트(IP 주소 기준)가 정해진 시간 동안 보낼 수 있는 요청의 수를 제한합니다. 예를 들어, '1분에 5번'으로 제한하면, 6번째 요청은 `429 Too Many Requests` 오류를 반환하며 자동으로 차단됩니다.

In [ ]:
%%writefile -a mcp_server.py

from slowapi import Limiter, _rate_limit_exceeded_handler
from slowapi.util import get_remote_address
from slowapi.errors import RateLimitExceeded

# --- 안정성: 요청 수 제한 ---
limiter = Limiter(key_func=get_remote_address)
app.state.limiter = limiter
app.add_exception_handler(RateLimitExceeded, _rate_limit_exceeded_handler)

# Rate limiting이 적용된 최종 엔드포인트들
@app.post("/api/v1/tools/web_search", summary="웹 검색 (최종)", tags=["Production Tools"])
@limiter.limit("10/minute")
async def production_web_search(request: Request, data: ToolCallRequest, api_key: str = Depends(get_api_key)):
    result = await web_search(data.query)
    return {"tool": "web_search", "query": data.query, "result": result}

@app.post("/api/v1/tools/news_api", summary="뉴스 검색 (최종)", tags=["Production Tools"])
@limiter.limit("10/minute")
async def production_news_search(request: Request, data: ToolCallRequest, api_key: str = Depends(get_api_key)):
    result = await news_api_search(data.query)
    return {"tool": "news_api", "query": data.query, "result": result}

@app.post("/api/v1/tools/arxiv_search", summary="논문 검색 (최종)", tags=["Production Tools"])
@limiter.limit("10/minute")
async def production_arxiv_search(request: Request, data: ToolCallRequest, api_key: str = Depends(get_api_key)):
    result = await arxiv_search(data.query)
    return {"tool": "arxiv_search", "query": data.query, "result": result}

print("✅ Rate limiting이 적용된 최종 프로덕션 엔드포인트들이 추가되었습니다.")

### 9. 최종 서버 재실행 및 보안/안정성 테스트

이제 인증과 요청 수 제한이라는 강력한 갑옷을 두른 최종 버전의 MCP 서버를 실행합니다. 그리고 이 보안 기능들이 실제로 잘 작동하는지, 다양한 시나리오를 통해 직접 테스트하여 검증합니다.

In [ ]:
# 서버 재실행
server_url, server_process = launch_fastapi_local("mcp_server.py", 8501)

if server_url:
    print(f"\n최종 보안 서버가 성공적으로 재시작되었습니다. Local URL: {server_url}")
    import time

    time.sleep(5)

In [ ]:
import requests
import time

if "server_url" in locals() and server_url:
    master_key = os.environ["MASTER_API_KEY"]
    headers = {"X-API-Key": master_key}
    wrong_headers = {"X-API-Key": "wrong-key"}

    # --- 테스트 1: API 키 인증 테스트 ---
    print("\n--- API 키 인증 테스트 시작 ---")

    # 보안 엔드포인트 테스트
    try:
        # (1-1) 잘못된 키로 요청
        response_fail = requests.get(f"{server_url}/secure/resources/style_guides/default", headers=wrong_headers)
        print(f"(1-1) 잘못된 키: Status {response_fail.status_code}")
        if response_fail.status_code == 401:
            print("    인증 실패 정상 처리")

        # (1-2) 올바른 키로 요청
        response_success = requests.get(f"{server_url}/secure/resources/style_guides/default", headers=headers)
        print(f"(1-2) 올바른 키: Status {response_success.status_code}")
        if response_success.status_code == 200:
            print("    인증 성공")
            result = response_success.json()
            print(f"    응답: {result['content'][:50]}...")

    except Exception as e:
        print(f"테스트 중 오류 발생: {e}")

    # --- 테스트 2: 프로덕션 엔드포인트 테스트 ---
    print("\n--- 프로덕션 엔드포인트 테스트 ---")
    try:
        test_data = {"query": "FastAPI"}
        response = requests.post(f"{server_url}/api/v1/tools/web_search", json=test_data, headers=headers)
        print(f"프로덕션 API 테스트: Status {response.status_code}")
        if response.status_code == 200:
            print("    프로덕션 API 정상 작동")

    except Exception as e:
        print(f"프로덕션 API 테스트 실패: {e}")

else:
    print("서버 URL이 생성되지 않았습니다.")

# [Lv4-Day4-Lab2] The System of Systems: A Multi-Framework Blog Agent

### 실습 목표
4일차의 두 번째 실습에서는, 3일차에 개별적으로 완성했던 각기 다른 프레임워크의 Agent들을 하나의 거대한 **'통합 지능 시스템'**으로 엮어냅니다. 우리는 각 프레임워크의 장점을 극대화하는, 매우 실무적인 **분산 시스템 아키텍처**를 설계하고 구현합니다. `LangGraph`가 총괄 지휘를, `CrewAI`가 콘텐츠 생성을, `ADK`가 품질 검증을 담당하며, 이 모든 소통은 Lab 1에서 만든 `MCP 서버`를 통해 이루어집니다.

이번 **Part 1**에서는 이 거대한 시스템의 '두뇌'이자 '지휘자' 역할을 할 **`LangGraph` 기반 오케스트레이터**의 기반을 다지는 데 집중합니다.

1.  **MCP 서버 클라이언트 구현:** Lab 1에서 만든 MCP 서버와 안정적으로 통신하기 위한 전문 Python 클라이언트(`MCPClient`)를 구현합니다. 이는 서비스 지향 아키텍처에서 API와 상호작용하는 표준적인 방식입니다.
2.  **'팀을 Tool로' 추상화:** `CrewAI` 콘텐츠 제작팀과 `ADK` 품질 검증 Agent 전체를, 오케스트레이터가 호출할 수 있는 하나의 강력한 **'Tool'**로 추상화하는 고급 설계 패턴을 학습합니다.
3.  **오케스트레이터 뼈대 구축:** 전체 블로그 생성 워크플로우를 관리할 `LangGraph`의 상태(State)와 핵심 노드(Node)들의 기본 구조를 정의하여, 이어질 파트에서 구체적인 로직을 채워나갈 준비를 합니다.

### 0. 사전 준비: 라이브러리 설치 및 환경 설정
이 실습은 3일차와 4일차 Lab 1에서 사용했던 모든 기술 스택을 요구합니다. 필요한 라이브러리들을 설치하고, MCP 서버와 통신하기 위한 `MASTER_API_KEY`를 포함한 모든 API 키를 설정합니다.

In [ ]:
import os
from getpass import getpass

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Google API 키를 입력하세요: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass("Tavily API 키를 입력하세요: ")

if "NEWS_API_KEY" not in os.environ:
    os.environ["NEWS_API_KEY"] = getpass("News API 키를 입력하세요: ")

# 마스터 키는 코드에 직접 할당합니다.
os.environ["MASTER_API_KEY"] = "samsung-llm-agent-lv4-master-key"

print("✅ 모든 API 키가 성공적으로 설정되었습니다.")

### 1. MCP 서버와의 통신 채널: `MCPClient` 구현

가장 먼저, 우리의 새로운 오케스트레이터 Agent가 Lab 1에서 만든 MCP 서버와 안전하고 편리하게 통신할 수 있도록 전문 **클라이언트 클래스**를 만듭니다. 이 클래스는 API 요청의 복잡한 세부 사항(URL 조합, 헤더 설정, 에러 처리 등)을 캡슐화하여, 메인 로직에서는 `mcp_client.call_tool(...)`와 같이 매우 간결하게 서버의 기능을 호출할 수 있게 해줍니다.


In [ ]:
import requests
import json

# 아래 변수는 Lab 1에서 생성된 MCP 서버의 server URL입니다.
# server_url= ""

mcp_server_url = server_url


class MCPClient:
    def __init__(self, base_url: str, api_key: str):
        if not base_url:
            raise ValueError("MCP 서버의 base_url이 필요합니다.")
        self.base_url = base_url
        self.headers = {"X-API-Key": api_key, "Content-Type": "application/json"}

    def call_tool(self, tool_name: str, query: str) -> dict:
        """MCP 서버의 Tool 엔드포인트를 호출합니다."""
        endpoint = f"{self.base_url}/tools/{tool_name}"
        payload = {"query": query}
        try:
            response = requests.post(endpoint, headers=self.headers, json=payload, timeout=120)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            return {"error": f"HTTP 오류: {e.response.status_code}", "detail": e.response.text}
        except Exception as e:
            return {"error": f"Tool 호출 중 오류 발생: {e}"}

    def get_resource(self, resource_type: str, resource_id: str) -> dict:
        """MCP 서버의 리소스 엔드포인트를 호출합니다."""
        endpoint = f"{self.base_url}/resources/{resource_type}/{resource_id}"
        try:
            response = requests.get(endpoint, headers=self.headers, timeout=30)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            return {"error": f"HTTP 오류: {e.response.status_code}", "detail": e.response.text}
        except Exception as e:
            return {"error": f"리소스 조회 중 오류 발생: {e}"}


# 클라이언트 인스턴스화 및 테스트
if mcp_server_url:
    mcp_client = MCPClient(base_url=mcp_server_url, api_key=os.environ["MASTER_API_KEY"])
    print("✅ MCPClient가 성공적으로 초기화되었습니다.")
    # 서버 상태 확인 테스트
    test_response = requests.get(mcp_server_url)
    print(f"   서버 상태: {test_response.json().get('message', '연결 실패')}")
else:
    print("❌ MCP 서버 URL을 입력해주세요. Lab 1 실습을 먼저 실행해야 합니다.")
    mcp_client = None

### 2. '팀을 Tool로' 추상화: 오케스트레이터를 위한 전문가 Tool 정의
이제 우리 `LangGraph` 오케스트레이터가 사용할 Tool들을 정의합니다. 여기서 가장 중요한 것은, 단순한 Tool(`web_search` 등) 외에, **다른 Agent 시스템 전체를 하나의 Tool처럼 호출**하는 고수준의 추상화를 구현하는 것입니다.

이 파트에서는 실제 `CrewAI`와 `ADK` 로직을 구현하기 전에, 이들의 역할을 **가짜(Mock) 함수**로 먼저 정의합니다. 이는 전체 워크플로우의 뼈대를 먼저 잡고 각 부분을 독립적으로 개발할 수 있게 해주는 중요한 개발 전략입니다.

In [ ]:
from langchain.tools import tool
from pydantic import BaseModel, Field


# --- 입력 스키마 정의 ---
class ContentCreationInput(BaseModel):
    topic: str = Field(description="콘텐츠 제작팀에게 전달할 블로그 글의 핵심 주제")
    user_preferences: str = Field(description="사용자의 스타일 선호도나 특별 요청사항")


class QualityAssuranceInput(BaseModel):
    topic: str = Field(description="품질 검증을 수행할 블로그 글의 주제")
    draft_content: str = Field(description="검증할 블로그 글의 초안 내용")


# --- '팀'을 대표하는 Tool 함수 (Mock 버전) ---
@tool(args_schema=ContentCreationInput)
def run_crewai_content_creation(topic: str, user_preferences: str) -> str:
    """'콘텐츠 제작 전문팀(CrewAI)'을 호출하여, 주어진 주제와 사용자 선호도에 맞는 블로그 글의 초안을 작성하도록 지시합니다. 리서치부터 초안 작성까지의 모든 과정을 이 팀이 자율적으로 수행합니다."""
    print(f"\n--- [Tool Call] ➡️  CrewAI 팀에게 '{topic}' 주제로 콘텐츠 제작 요청 ---")
    # (Part 6에서 이 부분에 실제 CrewAI 실행 로직이 구현됩니다)
    mock_draft = f"## {topic}에 대한 심층 분석\n\n이것은 CrewAI 팀이 작성한 초안입니다. 사용자의 선호도({user_preferences})를 반영했습니다..."
    print(f"--- [Tool Result] ⬅️  CrewAI 팀이 초안을 반환함 ---")
    return mock_draft


@tool(args_schema=QualityAssuranceInput)
def run_adk_quality_assurance(topic: str, draft_content: str) -> str:
    """'품질 보증 전문팀(ADK)'을 호출하여, 작성된 초안의 품질을 검증하고 최종 편집을 수행하도록 지시합니다. SEO, 가독성 등을 종합적으로 평가합니다."""
    print(f"\n--- [Tool Call] ➡️  ADK 팀에게 초안 품질 검증 요청 ---")
    # (Part 6에서 이 부분에 실제 ADK Agent 실행 로직이 구현됩니다)
    mock_qa_report = f"## 품질 검증 보고서\n- SEO 점수: 85/100\n- 가독성: 양호\n- 최종 편집본: {draft_content[:50]}... [최종 편집 완료]"
    print(f"--- [Tool Result] ⬅️  ADK 팀이 최종 편집본과 보고서를 반환함 ---")
    return mock_qa_report


# 오케스트레이터가 사용할 전체 Tool 리스트
orchestrator_tools = [run_crewai_content_creation, run_adk_quality_assurance]

print("✅ 오케스트레이터가 사용할 전문가 팀 Tool들이 (Mock 버전으로) 성공적으로 정의되었습니다.")

### 3. 오케스트레이터의 '기억': 상태(State) 정의

이제 전체 워크플로우의 '중앙 프로젝트 관리 보드' 역할을 할 `LangGraph`의 상태(State)를 정의합니다. 이 `OrchestratorState`는 사용자의 초기 요청부터 최종 결과물이 나올 때까지 모든 핵심 데이터와 중간 산출물을 추적하고 관리합니다. 각 필드는 우리 Multi-Framework 팀의 작업 단계를 명확하게 나타냅니다.

In [ ]:
from typing import TypedDict, Annotated, List, Optional
import operator


class OrchestratorState(TypedDict):
    """Multi-Framework 워크플로우의 전체 상태를 관리합니다."""

    topic: str  # 사용자가 제공한 초기 블로그 주제
    user_preferences: str  # 사용자의 스타일 및 요구사항
    initial_draft: Optional[str] = None  # CrewAI 팀이 생성한 초안
    qa_report: Optional[str] = None  # ADK 팀이 생성한 품질 검증 보고서
    final_post: Optional[str] = None  # ADK 팀이 최종 편집한 블로그 글

    # 워크플로우의 진행 상황을 추적하기 위한 로그
    flow_log: Annotated[List[str], operator.add]


print("✅ LangGraph 오케스트레이터를 위한 OrchestratorState가 성공적으로 정의되었습니다.")

### 4. 오케스트레이터의 '행동': 핵심 노드(Node) 함수 구현

이제 워크플로우의 각 주요 단계를 수행할 '노드(Node)' 함수들을 구현합니다. 각 노드는 `OrchestratorState`를 입력받아, **자신이 직접 작업을 처리하는 것이 아니라, 이전에 정의한 전문가 Tool(즉, 다른 Agent 시스템)을 호출하여 작업을 '위임(Delegate)'**합니다. 이것이 바로 오케스트레이터의 핵심 역할입니다.

In [ ]:
def start_node(state: OrchestratorState) -> dict:
    """워크플로우를 시작하고 초기 로그를 남깁니다."""
    print("--- 🚀 Node: Starting Orchestration ---")
    return {"flow_log": [f"Orchestration started for topic: {state['topic']}"]}


def content_creation_node(state: OrchestratorState) -> dict:
    """CrewAI 콘텐츠 제작팀에게 초안 작성을 위임합니다."""
    print("--- ✍️  Node: Delegating to CrewAI Content Creation Team ---")
    topic = state["topic"]
    user_preferences = state["user_preferences"]

    draft = run_crewai_content_creation(topic, user_preferences)

    return {"initial_draft": draft, "flow_log": ["CrewAI team successfully generated the initial draft."]}


def quality_assurance_node(state: OrchestratorState) -> dict:
    """ADK 품질 보증팀에게 최종 편집을 위임합니다."""
    print("--- 🧐 Node: Delegating to ADK Quality Assurance Team ---")
    topic = state["topic"]
    draft_content = state["initial_draft"]

    qa_result = run_adk_quality_assurance(topic, draft_content)

    return {
        "qa_report": qa_result,
        "final_post": qa_result,
        "flow_log": ["ADK team successfully completed quality assurance and final editing."],
    }


print("✅ 오케스트레이터의 핵심 노드 3개가 성공적으로 정의되었습니다.")

### 5. 오케스트레이터의 '설계도': LangGraph 워크플로우 구축
이제 모든 부품(State, Nodes)이 준비되었습니다. 이들을 `LangGraph`를 사용하여 하나의 완전한 워크플로우 '설계도'로 조립합니다. 이번 파트에서는 **단순한 순차(Sequential) 워크플로우**를 먼저 구성하여, 전체 시스템이 올바르게 연동되는지 확인하는 데 집중합니다.

In [ ]:
from langgraph.graph import StateGraph, END

# 1. StateGraph 객체 생성
workflow = StateGraph(OrchestratorState)

# 2. 노드 추가
workflow.add_node("start", start_node)
workflow.add_node("create_content", content_creation_node)
workflow.add_node("assure_quality", quality_assurance_node)

# 3. 엣지 연결 (워크플로우 흐름 정의)
workflow.set_entry_point("start")
workflow.add_edge("start", "create_content")
workflow.add_edge("create_content", "assure_quality")
workflow.add_edge("assure_quality", END)

# 4. 그래프 컴파일
orchestrator_graph = workflow.compile()

print("✅ LangGraph 기반 오케스트레이터 워크플로우가 성공적으로 컴파일되었습니다.")

### 6. 오케스트레이터의 '영혼': 실제 Agent 팀 연동

이제 이번 실습의 가장 핵심적인 단계입니다. Part 1에서 우리가 Mock 함수로 정의했던 '팀' Tool들을, 3일차에 만들었던 실제 `CrewAI` 팀과 `ADK` Agent의 코드로 대체하여 생명을 불어넣습니다.

**핵심 통합 포인트:**
1.  **`CrewAI 팀 구현`**: 3일차 Lab 2에서 만든 계층적 팀의 코드를 `run_crewai_content_creation` 함수 안으로 가져옵니다. 여기서 중요한 것은, `Research Agent`가 더 이상 로컬 Tool을 사용하는 것이 아니라, 우리가 만든 **`MCPClient`를 통해 중앙 서버의 Tool을 호출**하도록 수정하는 것입니다. 이를 통해 `CrewAI` 시스템이 우리의 분산 아키텍처에 완벽하게 편입됩니다.
2.  **`ADK Agent 구현`**: 3일차 Lab 4에서 영감을 받은 ADK 기반 '비평가' Agent 로직을 `run_adk_quality_assurance` 함수 안에 구현합니다. 이 Agent 역시 필요하다면 `MCPClient`를 통해 외부 리소스(e.g., 스타일 가이드)에 접근할 수 있습니다.

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai.tools import BaseTool


# --- 1. CrewAI 팀의 Tool을 MCP 서버와 연동하기 위한 래퍼(Wrapper) 클래스 ---
class MCPTool(BaseTool):
    name: str
    description: str
    client: "MCPClient"

    def _run(self, query: str) -> str:
        print(f"  [MCP Bridge] CrewAI -> MCP Server: Calling tool '{self.name}' with query '{query}'")
        response = self.client.call_tool(self.name, query)
        return json.dumps(response, ensure_ascii=False)


# --- 2. 실제 CrewAI 팀 실행 로직으로 함수 재구현 ---
def run_crewai_content_creation(topic: str, user_preferences: str) -> str:
    """'콘텐츠 제작 전문팀(CrewAI)'을 호출하여, 주어진 주제와 사용자 선호도에 맞는 블로그 글의 초안을 작성하도록 지시합니다."""
    print(f"\n--- [Orchestrator] ➡️  CrewAI 팀에게 '{topic}' 주제로 콘텐츠 제작 요청 ---")

    if not mcp_client:
        return "오류: MCPClient가 초기화되지 않았습니다."

    try:
        # MCP 서버의 Tool을 사용하는 CrewAI 전용 Tool 생성
        crew_web_search = MCPTool(name="web_search", description="웹에서 최신 정보를 검색합니다.", client=mcp_client)

        # CrewAI Agent와 Task 정의
        llm = LLM(model="gemini/gemini-2.5-flash-lite", temperature=0.1)

        researcher = Agent(
            role="전문 리서처",
            goal=f"{topic}에 대한 최신이고 정확한 정보 수집",
            backstory="10년 경력의 기술 전문 리서처로, 복잡한 기술을 쉽게 설명하는 것을 전문으로 합니다.",
            tools=[crew_web_search],
            llm=llm,
            verbose=True,
        )

        writer = Agent(
            role="전문 작성가",
            goal="리서치 결과를 바탕으로 읽기 쉬운 블로그 글 작성",
            backstory="기술 블로그 작성 전문가로, 어려운 내용을 일반인도 이해할 수 있게 글쓰기를 전문으로 합니다.",
            llm=llm,
            verbose=True,
        )

        # Task 정의
        research_task = Task(
            description=f"""
            주제: {topic}
            사용자 요구사항: {user_preferences}

            위 주제에 대해 웹 검색을 통해 최신 정보를 수집하고 분석하세요.
            특히 실제 사례와 구체적인 예시를 찾는 데 집중하세요.
            """,
            expected_output="주제에 대한 상세한 리서치 보고서 (최소 500자)",
            agent=researcher,
        )

        write_task = Task(
            description=f"""
            리서치 결과를 바탕으로 '{topic}'에 대한 블로그 글을 작성하세요.
            사용자 요구사항: {user_preferences}

            구조:
            - 매력적인 제목
            - 서론 (독자의 관심 유발)
            - 본론 (핵심 내용, 구체적 예시 포함)
            - 결론 (요약 및 미래 전망)
            """,
            expected_output="완성된 블로그 글 (제목 포함, 최소 1000자)",
            agent=writer,
            context=[research_task],
        )

        # Crew 실행
        crew = Crew(
            agents=[researcher, writer], tasks=[research_task, write_task], process=Process.sequential, verbose=True
        )

        result = crew.kickoff()
        print(f"--- [Orchestrator] ⬅️  CrewAI 팀이 초안을 반환함 ---")
        return str(result)

    except Exception as e:
        error_msg = f"CrewAI 실행 중 오류 발생: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg


# --- 3. 실제 ADK Agent 실행 로직으로 함수 재구현 ---
def run_adk_quality_assurance(topic: str, draft_content: str) -> str:
    """'품질 보증 전문팀(ADK)'을 호출하여, 작성된 초안의 품질을 검증하고 최종 편집을 수행하도록 지시합니다."""
    print(f"\n--- [Orchestrator] ➡️  ADK 팀에게 초안 품질 검증 요청 ---")

    try:
        # ADK 스타일의 품질 검증 및 편집
        qa_llm = LLM(model="gemini/gemini-2.5-flash-lite", temperature=0.1)

        prompt = f"""
        당신은 수석 편집자입니다. 다음 블로그 초안을 검토하고 최종 발행본으로 만들어주세요.

        **주제**: {topic}

        **초안**:
        {draft_content}

        **검토 및 편집 요청사항**:
        1. 문법 및 맞춤법 확인
        2. 논리적 구조 개선
        3. 가독성 향상 (문단 나누기, 소제목 추가 등)
        4. SEO 친화적 제목 및 내용 최적화
        5. 결론 부분 강화

        최종 편집본을 Markdown 형식으로 제공해주세요.
        """

        print("--- [Debug] ADK LLM 호출을 시작합니다. 입력 글자 수:", len(draft_content), "---")
        final_post = qa_llm.call(prompt)
        print("--- [Debug] ADK LLM 호출이 성공적으로 완료되었습니다. ---")  # 이 로그가 출력되는지 확인!

        qa_report = f"""## 품질 검증 보고서

### 검증 항목
- ✅ 문법 및 맞춤법: 검토 완료
- ✅ 논리적 구조: 개선 적용
- ✅ 가독성: 향상 처리
- ✅ SEO 최적화: 적용 완료

### 최종 편집본
{final_post}
"""

        print(f"--- [Orchestrator] ⬅️  ADK 팀이 최종 편집본과 보고서를 반환함 ---")
        return qa_report

    except Exception as e:
        error_msg = f"ADK 품질 검증 중 오류 발생: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg


print("✅ 오케스트레이터의 Tool들이 실제 Agent 팀의 실행 로직으로 업데이트되었습니다.")

### 7. 최종 실행: Multi-Framework 오케스트라의 연주

이제 모든 배우들이 무대에 올랐고, 각자의 역할을 정확히 인지했으며, 중앙 자원 허브(MCP 서버)와 소통할 준비를 마쳤습니다. `LangGraph` 지휘자는 지휘봉을 들었습니다.

아래 셀을 실행하여, 우리의 Multi-Framework 통합 시스템이 하나의 복잡한 블로그 생성 프로젝트를 처음부터 끝까지 자율적으로 수행하는 과정을 관찰해 봅시다. 실행 로그를 통해, `LangGraph` 오케스트레이터가 어떻게 `CrewAI` 팀과 `ADK` 팀에게 순차적으로 작업을 위임하고, 각 팀이 MCP 서버의 Tool을 호출하며 임무를 완수하는지 그 장대한 흐름을 확인할 수 있습니다.

In [ ]:
# 최종 워크플로우 실행
if mcp_client:
    # 1. 실행을 위한 초기 상태 정의
    initial_state = {
        "topic": "양자 컴퓨팅과 AI의 융합이 가져올 미래 기술 혁신",
        "user_preferences": "독자들이 쉽게 이해할 수 있도록, 전문 용어는 최소화하고 현실적인 예시를 많이 들어주세요.",
        "flow_log": [],
    }

    print("🚀" + "=" * 70)
    print(f"    Multi-Framework Blog Agent, '{initial_state['topic']}'에 대한 작업을 시작합니다.")
    print("=" * 73)

    # 2. LangGraph 스트림 실행
    final_state = None
    # .stream()을 사용하면 각 노드의 실행 결과를 실시간으로 확인할 수 있습니다.
    for state_update in orchestrator_graph.stream(initial_state, stream_mode="values"):
        final_state = state_update
        # 실시간 로그 출력 (전체 내용 대신 요약 정보 출력)
        print("\n--- [Orchestrator Log Summary] ---")
        for key, value in final_state.items():
            if isinstance(value, str) and value is not None:
                print(f'"{key}": (string, length: {len(value)})')
            else:
                print(f'"{key}": {value}')
        print("---------------------------------")

    # 3. 최종 결과물 출력
    print("\n" + "=" * 80)
    print("                           ✅ 최종 결과물 ✅")
    print("=" * 80)
    print(final_state.get("final_post", "최종 결과물이 생성되지 않았습니다."))
else:
    print("❌ MCP 서버가 실행되지 않았거나 클라이언트가 초기화되지 않았습니다. 이전 파트를 먼저 실행해주세요.")